In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.preprocessing import StandardScaler   
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, act_func):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(2, 1)
        self.act_func = act_func
        
    def forward(self, x):
        if (self.act_func == 'sigmoid'):
            return torch.sigmoid(self.fc(x))
        elif (self.act_func == 'tanh'):
            return (torch.tanh(self.fc(x)) + 1) / 2
        elif (self.act_func == 'atan'):
            return (torch.atan(self.fc(x)) + math.pi / 2) / math.pi
        else:
            return 0

In [ ]:
class Net(nn.Module):
    def __init__(self, n_neurons, hidden_act_func, final_act_func):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, n_neurons)
        self.fc2 = nn.Linear(n_neurons, 1)
        self.haf = hidden_act_func
        self.faf = final_act_func
        
    def forward(self, x):
        if (self.haf == 'sigmoid'):
            x = torch.sigmoid(self.fc1(x))
        elif (self.haf == 'tanh'):
            x = torch.tanh(self.fc1(x))
        elif (self.haf == 'atan'):
            x = torch.atan(self.fc1(x))
        elif (self.haf == 'relu'):
            x = torch.relu(self.fc1(x))
        else:
            print('gg')
            return 0
        
        if (self.faf == 'sigmoid'):
            return torch.sigmoid(self.fc2(x))
        elif (self.faf == 'tanh'):
            return (torch.tanh(self.fc2(x)) + 1) / 2
        elif (self.faf == 'atan'):
            return (torch.atan(self.fc2(x)) + math.pi / 2) / math.pi
        else:
            print('gg')
            return 0

In [ ]:
## train data
class Data(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
class Data_without_targets(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
def binary_acc(y_pred, y_test):
    correct_results_sum = (torch.round(y_pred) == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
def train(model, optimizer, data_loader):
    criterion = nn.BCELoss()
    model.train()
    for e in range(1, n_epochs + 1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in data_loader:
            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return [epoch_loss/len(data_loader), epoch_acc/len(data_loader)]

In [ ]:
def test(model, data_loader):
    criterion = nn.BCELoss()
    model.eval()
    test_loss = 0
    test_acc = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))        

            test_loss += loss.item()
            test_acc += acc.item()

    return [test_loss/len(data_loader), test_acc/len(data_loader)]

In [ ]:
def make_meshgrid(x, y, h=.02, margin=1):
    x_min, x_max = x.min() - margin, x.max() + margin
    y_min, y_max = y.min() - margin, y.max() + margin
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, model, xx, yy, **params):
    with torch.no_grad():
        model.eval()
        y_pred = model(torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float))
        y_pred = y_pred.detach().numpy()
        y_pred = y_pred.reshape(xx.shape)               
        out = ax.contourf(xx, yy, y_pred, **params)
    return out

def plot_results(ax, model, X, y, title):
    x1, x2 = (X.iloc[:, 0], X.iloc[:, 1])
    xx, yy = make_meshgrid(x1, x2, margin=margin)
    plot_contours(ax, model, xx, yy, cmap=plt.cm.RdYlGn, alpha=0.8)
    ax.scatter(x1, x2, c=y, cmap=plt.cm.RdYlGn, s=50, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('X 1')
    ax.set_ylabel('X 2')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

In [ ]:
n_epochs = 200
batch_size_0 = 10
batch_size_1 = 25
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [ ]:
data_0 = pd.read_csv('nn_0.csv').replace({'class' : {-1 : 0}})
X_0 = data_0.iloc[:, 0:-1]
y_0 = data_0.iloc[:, -1]

In [ ]:
data_1 = pd.read_csv('nn_1.csv').replace({'class' : {-1 : 0}})
X_1 = data_1.iloc[:, 0:-1]
y_1 = data_1.iloc[:, -1]

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(10, 5))
sns.scatterplot(x='X1', y='X2', data=data_0, hue='class', ax=axes[0])
sns.scatterplot(x='X1', y='X2', data=data_1, hue='class', ax=axes[1])
plt.tight_layout()

In [ ]:
X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(X_0, y_0, test_size=0.3, random_state=47)
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.3, random_state=47)

In [ ]:
train_data_0 = Data(torch.FloatTensor(X_0_train.values), torch.FloatTensor(y_0_train.values))
train_data_1 = Data(torch.FloatTensor(X_1_train.values), torch.FloatTensor(y_1_train.values))
test_data_0 = Data(torch.FloatTensor(X_0_test.values), torch.FloatTensor(y_0_test.values))
test_data_1 = Data(torch.FloatTensor(X_1_test.values), torch.FloatTensor(y_1_test.values))

In [ ]:
train_loader_0 = DataLoader(dataset=train_data_0, batch_size=batch_size_0, shuffle=True)
test_loader_0 = DataLoader(dataset=test_data_0, batch_size=batch_size_0)

train_loader_1 = DataLoader(dataset=train_data_1, batch_size=batch_size_1, shuffle=True)
test_loader_1 = DataLoader(dataset=test_data_1, batch_size=batch_size_1)

In [ ]:
margin = 0.2
plt.figure(figsize=(12, 15))

i = 1
act_funcs = ['sigmoid', 'tanh', 'atan']
optims = ['SGD', 'RMSProp', 'Adam']
test_avg_loss_list = []

for optim_element in optims:
    for act_func in act_funcs:
        perceptron = Perceptron(act_func)
        if (optim_element == 'SGD'):
            optimizer = optim.SGD(perceptron.parameters(), lr=learning_rate)
        elif (optim_element == 'RMSProp'):
            optimizer = optim.RMSprop(perceptron.parameters(), lr=learning_rate)
        elif (optim_element == 'Adam'):
            optimizer = optim.Adam(perceptron.parameters(), lr=learning_rate)
            
        [train_avg_loss, train_acc] = train(perceptron, optimizer, train_loader_0)
        [test_avg_loss, test_acc] = test(perceptron, test_loader_0)
        
        test_avg_loss_list.append(test_avg_loss)
        
        print(act_func, optim_element)
        print(f'Train: Avg.Loss: {train_avg_loss:.5f} | Acc: {train_acc:.3f}')
        print(f'Test: Avg.Loss: {test_avg_loss:.5f} | Acc: {test_acc:.3f}')
        print('____________________________________________________________')
        
        plot_results(plt.subplot(5, 3, i), perceptron, X_0, y_0, 
                     act_func + ' ' + optim_element + ', acc_score: ' + str(round(test_acc, 2)))
        i = i + 1
        
plt.tight_layout()

print('Test average loss = ' + str(round(np.average(np.array(test_avg_loss_list)), 3)))

In [ ]:
margin = 0.2
plt.figure(figsize=(12, 15))

i = 1
act_funcs = ['sigmoid', 'tanh', 'atan']
optims = ['SGD', 'RMSProp', 'Adam']
test_avg_loss_list = []
test_acc_list = []

for optim_element in optims:
    for act_func in act_funcs:
        perceptron = Perceptron(act_func)
        if (optim_element == 'SGD'):
            optimizer = optim.SGD(perceptron.parameters(), lr=learning_rate)
        elif (optim_element == 'RMSProp'):
            optimizer = optim.RMSprop(perceptron.parameters(), lr=learning_rate)
        elif (optim_element == 'Adam'):
            optimizer = optim.Adam(perceptron.parameters(), lr=learning_rate)
            
        [train_avg_loss, train_acc] = train(perceptron, optimizer, train_loader_1)
        [test_avg_loss, test_acc] = test(perceptron, test_loader_1)
        
        test_avg_loss_list.append(test_avg_loss)
        test_acc_list.append(test_acc)
        
        print(act_func, optim_element)
        print(f'Train: Avg.Loss: {train_avg_loss:.5f} | Acc: {train_acc:.3f}')
        print(f'Test: Avg.Loss: {test_avg_loss:.5f} | Acc: {test_acc:.3f}')
        print('____________________________________________________________')
        
        plot_results(plt.subplot(5, 3, i), perceptron, X_1, y_1, 
                     act_func + ' ' + optim_element + ', acc_score: ' + str(round(test_acc, 2)))
        i = i + 1
        
plt.tight_layout()
print('Test average loss = ' + str(round(np.average(np.array(test_avg_loss_list)), 3)))
print('Test average accuracy = ' + str(round(np.average(np.array(test_acc_list)), 3)))

In [ ]:
i = 1
n_neurons = range(2, 11)
hidden_act_funcs = ['sigmoid', 'tanh', 'atan', 'relu']
final_act_funcs = ['sigmoid', 'tanh', 'atan']
optims = ['SGD', 'RMSProp', 'Adam']
results = pd.DataFrame(columns=['n_neurons', 'optimizer', 'hidden_act_func', 'final_act_func', 'Test avg. loss', 'Test acc.'])

for n in n_neurons:
    for optim_element in optims:
        for hidden_act_func in hidden_act_funcs:
            for final_act_func in act_funcs:
                net = Net(n, hidden_act_func, final_act_func)
                if (optim_element == 'SGD'):
                    optimizer = optim.SGD(net.parameters(), lr=learning_rate)
                elif (optim_element == 'RMSProp'):
                    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate)
                elif (optim_element == 'Adam'):
                    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

                [train_avg_loss, train_acc] = train(net, optimizer, train_loader_1)
                [test_avg_loss, test_acc] = test(net, test_loader_1)

                results =   results.append({'n_neurons': n, 'optimizer': optim_element, 'hidden_act_func': hidden_act_func, 
                                            'final_act_func': final_act_func, 'Test avg. loss' : test_avg_loss,
                                            'Test acc.' : test_acc}, ignore_index=True)

                print(n, optim_element, hidden_act_func, final_act_func)
                print(f'Train: Avg.Loss: {train_avg_loss:.5f} | Acc: {train_acc:.3f}')
                print(f'Test: Avg.Loss: {test_avg_loss:.5f} | Acc: {test_acc:.3f}')
                print('____________________________________________________________')

                i = i + 1


In [ ]:
results.sort_values(['Test acc.', 'Test avg. loss'], ascending=[0, 1]).head(3)

In [ ]:
margin = 0.2
plt.figure(figsize=(15, 5))

n_neurons = [8, 7, 9]
hidden_act_funcs = ['tanh', 'atan', 'tanh']
final_act_funcs = ['tanh', 'tanh', 'sigmoid']
optim_element = 'RMSProp'

for i in range (0, 3):
    net = Net(n_neurons[i], hidden_act_funcs[i], final_act_funcs[i])
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate)
    train(net, optimizer, train_loader_1)
    plot_results(plt.subplot(1, 3, i + 1), net, X_1_test, y_1_test, 
                 'n_neurons = ' + str(n_neurons[i]) + '\n ' + optim_element 
                 + '\nhidden act. func = ' + hidden_act_funcs[i] + '\n final act. func = ' + final_act_funcs[i])

In [ ]:
X_1_train